# Parameter Optimization

Run grid search or random search optimization to find optimal parameters.

**Version**: v1.11.0  
**Architecture**: Modular (lib/optimize/, lib/config/, lib/paths/)

## Configuration

Define the parameter grid and optimization settings below.

**Note**: This notebook uses the v1.11.0 modular architecture. All imports use canonical paths from `lib/` packages.


In [ ]:
# Configuration
strategy_name = 'spy_sma_cross'
start_date = '2020-01-01'
end_date = None
method = 'grid'  # 'grid' or 'random'
objective = 'sharpe'  # 'sharpe', 'sortino', 'total_return', 'calmar'
train_pct = 0.7  # Training data percentage

# Parameter grid (for grid search)
param_grid = {
    'strategy.fast_period': [5, 10, 15, 20],
    'strategy.slow_period': [30, 50, 100],
}

# For random search, use distributions:
# param_distributions = {
#     'strategy.fast_period': [5, 10, 15, 20],
#     'strategy.slow_period': [30, 50, 100],
# }
n_iter = 100  # For random search


In [ ]:
# Setup
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

# Standard library imports
import json

# Third-party imports
import pandas as pd
import numpy as np

# Local imports - lib modules (v1.11.0 modular architecture)
# Optimization modules provide grid search, random search, and overfit detection
from lib.paths import get_project_root, get_results_dir
from lib.optimize import grid_search, random_search, split_data
from lib.config import load_strategy_params


In [ ]:
# Pre-optimization Validation
from lib.validation import validate_bundle
from lib.bundles import list_bundles, get_bundle_symbols
from lib.calendars import get_calendar_for_asset_class

# Check strategy parameters
try:
    params = load_strategy_params(strategy_name)
    print(f"✓ Strategy '{strategy_name}' loaded successfully")
    
    # Get asset class and calendar
    asset_class = params.get('strategy', {}).get('asset_class', 'equities')
    calendar_name = get_calendar_for_asset_class(asset_class)
    print(f"✓ Asset class: {asset_class}")
    print(f"✓ Trading calendar: {calendar_name}")
    
    # Validate parameter grid keys exist in strategy
    strategy_params = params.get('strategy', {})
    for param_key in param_grid.keys():
        param_path = param_key.split('.')
        if len(param_path) == 2:
            section, key = param_path
            if section in params and key in params[section]:
                print(f"✓ Parameter '{param_key}' found")
            else:
                print(f"⚠ Parameter '{param_key}' not found in strategy config")
    
except FileNotFoundError:
    print(f"⚠ Strategy '{strategy_name}' not found")
    print(f"  Ensure strategy exists before optimization")
except Exception as e:
    print(f"⚠ Parameter validation error: {e}")

# Check bundle availability
print("\nPre-optimization checks:")
print(f"  Strategy: {strategy_name}")

# List available bundles for reference
available_bundles = list_bundles()
print(f"  Available bundles: {len(available_bundles)}")

# Validate bundle if specified (bundle variable may be set in config)
# Note: bundle validation would require bundle name from strategy config

In [ ]:
# Run optimization
try:
    if method == 'grid':
        print(f"Running grid search for {strategy_name}...")
        print(f"Parameter grid: {len(param_grid)} parameters")
        results_df = grid_search(
            strategy_name=strategy_name,
            param_grid=param_grid,
            start_date=start_date,
            end_date=end_date,
            objective=objective,
            train_pct=train_pct
        )
    else:
        print(f"Running random search for {strategy_name}...")
        print(f"Iterations: {n_iter}")
        results_df = random_search(
            strategy_name=strategy_name,
            param_distributions=param_grid,  # Use same grid as distributions
            n_iter=n_iter,
            start_date=start_date,
            end_date=end_date,
            objective=objective,
            train_pct=train_pct
        )
    
    print(f"\n✓ Optimization complete: {len(results_df)} combinations tested")
    
except Exception as e:
    print(f"✗ Optimization failed: {e}")
    print(f"  Check that strategy exists and parameters are valid")
    raise


In [ ]:
# Display optimization results location
results_base = get_results_dir() / strategy_name / 'latest'
print(f"\nResults saved to: {results_base}")
print(f"  Absolute path: {results_base.resolve()}")

# List available result files
if results_base.exists():
    result_files = list(results_base.glob('*'))
    print(f"\nAvailable result files ({len(result_files)}):")
    for f in sorted(result_files)[:5]:  # Show first 5
        print(f"  - {f.name}")

In [ ]:
# Display results
test_obj_col = f'test_{objective}'
if test_obj_col in results_df.columns:
    best_idx = results_df[test_obj_col].idxmax()
    best_row = results_df.loc[best_idx]
    
    print("=" * 60)
    print("BEST PARAMETERS")
    print("=" * 60)
    for param_name in param_grid.keys():
        if param_name in best_row:
            print(f"{param_name}: {best_row[param_name]}")
    
    print(f"\nPerformance:")
    print(f"  Train {objective}: {best_row.get(f'train_{objective}', 0):.4f}")
    print(f"  Test {objective}: {best_row.get(test_obj_col, 0):.4f}")
    print(f"  Train Sharpe: {best_row.get('train_sharpe', 0):.4f}")
    print(f"  Test Sharpe: {best_row.get('test_sharpe', 0):.4f}")
    print("=" * 60)


In [ ]:
# Display top 10 results
print("\nTop 10 Results:")
print(results_df.nlargest(10, test_obj_col)[['train_' + objective, test_obj_col, 'train_sharpe', 'test_sharpe'] + list(param_grid.keys())])


In [ ]:
# Load overfit score
results_base = get_results_dir() / strategy_name / 'latest'
overfit_file = results_base / 'overfit_score.json'

if overfit_file.exists():
    with open(overfit_file) as f:
        overfit = json.load(f)
    
    print("\n" + "=" * 60)
    print("OVERFIT ANALYSIS")
    print("=" * 60)
    print(f"Efficiency (OOS/IS): {overfit.get('efficiency', 0):.3f}")
    print(f"PBO: {overfit.get('pbo', 0):.2f}")
    print(f"Verdict: {overfit.get('verdict', 'unknown')}")
    print("=" * 60)
else:
    print(f"⚠ Overfit score not found at: {overfit_file}")
    print(f"  Run optimization with overfit detection enabled")

In [ ]:
# Display heatmap if 2 parameters
if len(param_grid) == 2:
    from IPython.display import Image, display
    
    heatmap_file = results_base / f'heatmap_{objective}.png'
    
    if heatmap_file.exists():
        display(Image(str(heatmap_file)))
    else:
        print("⚠ Heatmap not available")
        print(f"  Expected at: {heatmap_file}")
        print(f"  Heatmaps are generated for 2-parameter optimizations")
else:
    print(f"ℹ Heatmap requires exactly 2 parameters (found {len(param_grid)})")
